# 1. Create YoloV3 Inference Engine
In this section, we will create an inference engine wrapper, a class that will get an image data as input, analyse it and return analysis result.

## 1.1. Get global variables
We will read the previously stored variables. We need the name of the directory that we will use to store in our ml solution files. We create a directory with the specified directory name (if not already exist)

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

isSolutionPath = get_key(envPath, "isSolutionPath")

import os
if not os.path.exists(isSolutionPath):
    os.mkdir(isSolutionPath)

## 5.2. Download ONNX ML model
Download sample ONNX Model to use in the solution. You can customize it with your own ONNX model and solution.

In [ ]:
# dont change below values as it will be embedded into score.py file...
# if changed, must also update the score.py content according to new file names
onnxModelFileName = "model.onnx"
onnxLabelFileName = "labels.txt"

onnxModelUrl = "https://media.githubusercontent.com/media/onnx/models/master/vision/object_detection_segmentation/yolov3/model/yolov3-10.onnx"
onnxModelLabels = "https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/model_data/coco_classes.txt"

Below code snipped downloads Yolo V3 model from [ONNX Model zoo](https://github.com/onnx/models). Model download URLs frequently changes, so if below code fails, please update the source URLs accordingly.

In [ ]:
# Download the Model files
import urllib.request
import os

# Download the Tiny Yolo V3 pre-trained model
isSolutionModelFilePath = os.path.join(isSolutionPath, onnxModelFileName)
if not os.path.exists(isSolutionModelFilePath):
    res = urllib.request.urlretrieve(onnxModelUrl, isSolutionModelFilePath)
    print("Model file downloaded at: {}".format(isSolutionModelFilePath))
else:
    print("{} already exists here, so not downloading again.".format(isSolutionModelFilePath))
    
# Download the labels of the Yolo V3 pre-trained model
aixSolutionModelLabelFilePath = os.path.join(isSolutionPath, onnxLabelFileName)
if not os.path.exists(isSolutionModelLabelFilePath):
    res = urllib.request.urlretrieve(onnxModelLabels, isSolutionModelLabelFilePath)
    print("Labels file downloaded at: {}".format(isSolutionModelLabelFilePath))
else:
    print("{} already exists here, so not downloading again.".format(isSolutionModelLabelFilePath))

## 5.3. Create Inference Engine Wrapper
Here we create a class that will have different properties and methods to help scoring, analysing an image data. This class will also help us to specify analytics compute target such as CPU, VPU, FPGA etc. and also debugging features.  

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span> Specific to this sample, we are using Yolo V3 model. As you can see from the code below:  
- Yolo V3 model accepts only raw image bytes with 416 by 416 size.  
- We statically coded the image size (416x416) into the code. Because we expect the SCORE method to receive raw bytes in this size. If it is not 416x416 float32, than the code will crash
- Why? LVA sends video frames to the Score endpoint. LVA can send any image size and format. Since LVA can send image with 416x416 size, why we need to spend additional compute cycles here for re-sizing an image?

In [ ]:
%%writefile $isSolutionPath/score.py
import threading
import cv2
import numpy as np
import io
import onnxruntime
import json
import logging
import os
import linecache
import sys

logging.basicConfig(level=logging.DEBUG)

def PrintGetExceptionDetails():
    exType, exValue, exTraceback = sys.exc_info()

    tbFrame = exTraceback.tb_frame
    lineNo = exTraceback.tb_lineno
    fileName = tbFrame.f_code.co_filename

    linecache.checkcache(fileName)
    line = linecache.getline(fileName, lineNo, tbFrame.f_globals)

    exMessage = '[AIX] Exception:\n\tFile name: {0}\n\tLine number: {1}\n\tLine: {2}\n\tValue: {3}'.format(fileName, lineNo, line.strip(), exValue)

    logging.info(exMessage)


class MLModel:
    def __init__(self):
        try:
            self._modelFileName = 'model.onnx'
            self._labelFileName = 'labels.txt'
            self._lock = threading.Lock()

            with open(self._labelFileName, "r") as f:
                self._labelList = [l.rstrip() for l in f]
            
            self._onnxSession = onnxruntime.InferenceSession(self._modelFileName)

        except:
            PrintGetExceptionDetails()

    def Preprocess(self, cvImage):
        try:
            imageBlob = cv2.cvtColor(cvImage, cv2.COLOR_BGR2RGB)
            imageBlob = np.array(imageBlob, dtype='float32')
            imageBlob /= 255.
            imageBlob = np.transpose(imageBlob, [2, 0, 1])
            imageBlob = np.expand_dims(imageBlob, 0)

            return imageBlob
        except:
            PrintGetExceptionDetails()

    def Postprocess(self, boxes, scores, indices):
        try:
            detectedObjects = []

            for idx in indices:
                idxTuple = (idx[0], idx[2])
                temp = [i for i in boxes[idxTuple]]  # temp[1, 0, 3, 2] = xmin, ymin, xmax, ymax
                dobj = {
                    "type" : "entity",
                    "entity" : {
                        "tag" : {
                            "value" : self._labelList[idx[1]],
                            "confidence" : str(scores[tuple(idx)])
                        },
                        "box" : {
                            "l" : str(temp[1] / 416),
                            "t" : str(temp[0] / 416),
                            "w" : str((temp[3] - temp[1]) / 416),
                            "h" : str((temp[2] - temp[0]) / 416)
                        }
                    }
                }
                detectedObjects.append(dobj)

            return detectedObjects
        except:
            PrintGetExceptionDetails()

    def Score(self, cvImage):
        try:
            with self._lock:
                imageBlob = self.Preprocess(cvImage)
                boxes, scores, indices = self._onnxSession.run(None, {"input_1": imageBlob, "image_shape":np.array([[416, 416]], dtype=np.float32)})
        
            return self.Postprocess(boxes, scores, indices)

        except:
            PrintGetExceptionDetails()

    def About(self):
        return str("<H1>ONNX Version: " + onnxruntime.__version__ + "</H1><BR><H1> App version: v 1.0</H1>")

Score method of the above InferenceEngine class will return a dictionary of inferences in the following form:

```
        {
            "entity": {
                "box": {
                    "h": 0.3498992351271351,
                    "l": 0.027884870008988812,
                    "t": 0.6497463818662655,
                    "w": 0.212033897746693
                },
                "tag": {
                    "confidence": 0.9857677221298218,
                    "value": "person"
                }
            },
            "type": "entity"
        },
        {
            "entity": {
                "box": {
                    "h": 0.3593513820482337,
                    "l": 0.6868949751420454,
                    "t": 0.6334065123374417,
                    "w": 0.26539528586647726
                },
                "tag": {
                    "confidence": 0.9851594567298889,
                    "value": "person"
                }
            },
            "type": "entity"
        }
```